In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/30.TecDoc'
else:
    print('fuck')

path_root

'.'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(300)

from gevent.lock import BoundedSemaphore
semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import shutil
directory_out = os.path.join(path_root, './file/2.part')
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs

import requests
import json

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

import time

# = = =  = = =  = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = =  = = =  = = =

print('Data Loading...')
list_series_error, df_input = [], pd.read_excel(os.path.join(path_root, './file/1.page.xlsx'),
                                                header=0,
                                                dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

# = = =  = = =  = = =

def crawler(crawler_tuple):
    global api, list_series_error, crawler_remain

    crawler_index, crawler_series = crawler_tuple

    # = = =  = = =  = = =

    try:
        url_parse = urlparse(crawler_series['Url'])

        dict_param = parse_qs(url_parse.query)

        list_dict_correct = []
        for page in range(1, crawler_series['Page'] + 1):
            data = {'getArticles': {'applyDqmRules': True,
                                    'articleCountry': 'DE',
                                    'articleStatusIds': [],
                                    'criteriaFilters': [],
                                    'dataSupplierIds': [int(dict_param['brands'][0].strip())],
                                    'filterQueries': ['(dataSupplierId NOT IN (4978,4982))'],
                                    'genericArticleIds': [int(dict_param['groups'][0].strip())],
                                    'includeAccessoryArticles': True,
                                    'includeAll': True,
                                    'includeArticleCriteria': True,
                                    'includeArticleLogisticsCriteria': True,
                                    'includeArticleStatusFacets': True,
                                    'includeArticleText': True,
                                    'includeComparableNumbers': True,
                                    'includeCriteriaFacets': True,
                                    'includeDataSupplierFacets': True,
                                    'includeGTINs': True,
                                    'includeGenericArticleFacets': True,
                                    'includeGenericArticles': True,
                                    'includeImages': True,
                                    'includeLinkages': True,
                                    'includeLinks': True,
                                    'includeMisc': True,
                                    'includeOEMNumbers': True,
                                    'includePDFs': True,
                                    'includePartsListArticles': True,
                                    'includePrices': True,
                                    'includeReplacedByArticles': True,
                                    'includeReplacesArticles': True,
                                    'includeTradeNumbers': True,
                                    'lang': 'qa',
                                    'linkagesPerPage': 100,
                                    'page': page,
                                    'perPage': 1_000,
                                    'provider': 23365,
                                    'sort': [{'field': 'articleNumber', 'direction': 'asc'}]}}

            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_check = f'''https://web.tecalliance.net/tecdoc/qa/parts/search?brands={dict_param['brands'][0].strip()}&groups={dict_param['groups'][0].strip()}&page={page}'''

                    resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                         data=json.dumps(data),
                                         headers=get_header(**{'X-Api-Key': api}),
                                         proxies=get_proxy(),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if dict_['status'] == 200:
                            break
                        elif dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                            print(f'fuck api {api}')
                            time.sleep(3)
                            with open('./api.txt', 'r') as api_txt:
                                api = api_txt.read()
                except requests.exceptions.ProxyError as e:
                    if '456 CN Client Forbidden' in str(e):
                        print('Long live the People\'s Republic of China!')
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            list_part = dict_['articles']

            # = = =  = = =  = = =

            for i, part in enumerate(list_part):

                dict_information, list_information = {}, ['\n'.join([text.strip() for text in list_text['text']]) for list_text in part['articleText']]
                for j, information in enumerate(list_information):
                    dict_information[str(j)] = {part['articleText'][j]['informationTypeDescription'].strip(): information}

                # = = =  = = =  = = =

                list_oem = [oem['articleNumber'].replace(' ', '') for oem in part['oemNumbers']]
                oem = ';'.join(list(dict.fromkeys(list_oem)))

                # = = =  = = =  = = =

                dict_key, list_criteria = {}, part['articleCriteria']
                for criteria in list_criteria:
                    key = criteria['criteriaAbbrDescription'].strip() if 'criteriaAbbrDescription' in criteria else criteria['criteriaDescription'].strip()
                    if key in dict_key:
                        dict_key[key] += 1
                    else:
                        dict_key[key] = 1

                dict_criteria, dict_count = {}, {}
                for j, criteria in enumerate(list_criteria):
                    key = criteria['criteriaAbbrDescription'].strip() if 'criteriaAbbrDescription' in criteria else criteria['criteriaDescription'].strip()
                    if dict_key[key] == 1:
                        dict_criteria[str(j)] = {key: f'''{criteria['formattedValue'].strip()} {criteria['criteriaUnitDescription'].strip()}''' if 'criteriaUnitDescription' in criteria else criteria['formattedValue'].strip()}
                    else:
                        if key in dict_count:
                            dict_count[key] += 1
                        else:
                            dict_count[key] = 1
                        dict_criteria[str(j)] = {f'{key}-{dict_count[key]}': f'''{criteria['formattedValue'].strip()} {criteria['criteriaUnitDescription'].strip()}''' if 'criteriaUnitDescription' in criteria else criteria['formattedValue'].strip()}

                # = = =  = = =  = = =

                dict_src, list_src = {}, part['images']
                for j, src in enumerate(list_src):
                    dict_src[str(j)] = src['imageURL3200'].strip()

                # = = =  = = =  = = =

                list_dict_correct.append({'No': page * 1_000 + i - 999,
                                          'Brand': part['mfrName'].strip(),
                                          'Product Group': part['genericArticles'][0]['genericArticleDescription'].strip(),
                                          'Part Number': part['articleNumber'].replace(' ', ''),
                                          'Aditional Description': part['misc']['additionalDescription'].strip() if 'additionalDescription' in part['misc'] else '',
                                          'GTIN/EAN': ';'.join([gtin.strip() for gtin in part['gtins']]),
                                          'Packing Unit': part['misc']['quantityPerPackage'] if 'quantityPerPackage' in part['misc'] else '',
                                          'Quantity Per Packaging Unit': part['misc']['quantityPerPartPerPackage'] if 'quantityPerPartPerPackage' in part['misc'] else '',
                                          'Status': part['misc']['articleStatusDescription'].strip() if 'articleStatusDescription' in part['misc'] else '',
                                          'Remanufactured': part['misc']['isRemanufacturedPart'] if 'isRemanufacturedPart' in part['misc'] else '',
                                          'Trade Number': ';'.join([trade_number.strip() for trade_number in part['tradeNumbers']]),
                                          'Mandatory Material Certification': part['misc']['hasMandatoryMaterialCertification'] if 'hasMandatoryMaterialCertification' in part['misc'] else '',
                                          'OEM': oem,
                                          'Picture': '',
                                          'Url': f'''https://web.tecalliance.net/tecdoc/qa/parts/{dict_param['brands'][0].strip()}/{part['articleNumber'].strip()}/detail?groups={dict_param['groups'][0].strip()}''',
                                          'Json_Src': json.dumps(dict_src),
                                          'Json_Information': json.dumps(dict_information),
                                          'Json_Criteria': json.dumps(dict_criteria),
                                          'Part Code': part['genericArticles'][0]['legacyArticleId']})

            # = = =  = = =  = = =

            if crawler_remain % 3 == 0:
                clear_output()
            print(f'''[第{page}页] - [请求{crawler_retry}次] - [{len(list_dict_correct)}/{crawler_series['SKU Count']}] - [剩余{crawler_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')


        # = = =  = = =  = = =

        print('Data Outputting...')
        df_correct = pd.DataFrame(list_dict_correct)
        df_correct.to_excel(os.path.join(directory_out, f'''crawler_{crawler_series['No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
        print('Output Finished !')
        print()
    except:
        crawler_series['Check_Url'] = url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = =  = = =  = = =

    with semaphore_remain:
        crawler_remain -= 1

# = = =  = = =  = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/page_error.xlsx', index=False)
    print('爬虫存在error')
    print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

Data Loading...
Loading Done !

总数量：2

[第1页] - [请求1次] - [666/666] - [剩余1条] - [2024/12/04 18:06:35] - 1. https://web.tecalliance.net/tecdoc/qa/parts/search?brands=21&groups=298&page=1

Data Outputting...
Output Finished !

[第1页] - [请求1次] - [1000/1381] - [剩余0条] - [2024/12/04 18:06:36] - 2. https://web.tecalliance.net/tecdoc/qa/parts/search?brands=30&groups=298&page=1

[第2页] - [请求1次] - [1381/1381] - [剩余0条] - [2024/12/04 18:06:37] - 2. https://web.tecalliance.net/tecdoc/qa/parts/search?brands=30&groups=298&page=2

Data Outputting...
Output Finished !

输出ing...

Done ~
